In [ ]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout


import keras
from keras.layers import Embedding
from keras.layers import Dense, Flatten, LSTM
from keras.layers import Input, GlobalMaxPool1D, Dropout
from keras.layers import Activation
from keras.models import Model, Sequential
from keras import optimizers

In [ ]:
print(tf.__version__)

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [ ]:
df1 = pd.read_csv('datasets/geo/data/full_dataset/goemotions_1.csv')

In [ ]:
df1.shape

In [ ]:
df2 = pd.read_csv('datasets/geo/data/full_dataset/goemotions_2.csv')

In [ ]:
df2.shape

In [ ]:
df3 = pd.read_csv('datasets/geo/data/full_dataset/goemotions_3.csv')

In [ ]:
df3.shape

In [ ]:
#concatenate the 3 datasets

In [ ]:
df = pd.concat([df1, df2, df3], axis=0)

In [ ]:
df.shape

In [ ]:
df.head(3)

In [ ]:
#Delete unwanted columns

In [ ]:
df.drop(['created_utc','id', 'author', 'subreddit', 'link_id', 'parent_id', 
         'rater_id', 'example_very_unclear'], 
        inplace=True, axis=1)

In [ ]:
df.head(3)

In [ ]:
# available emotions
emotions = []
for col in df.columns:
    emotions.append(col)
    
emotions_df = pd.DataFrame(emotions)

In [ ]:
emotions_df.head()

In [ ]:
import string
pucn = string.punctuation
stop = stopwords.words('english')

In [ ]:
def clean_sentences(sentence):
    url = re.compile(r'https?://\S+|www\.\S+')
    url_free = url.sub(r'', sentence)
    alpha = ''.join([word for word in url_free if not word.isdigit()])
    no_white_space = re.sub(r'\s+', ' ',  alpha)  
    no_non_words = re.sub('\w*\d\w*', '', no_white_space)
    html=re.compile(r'<.*?>')
    no_html = html.sub(r'',no_non_words)
    no_punct=[words for words in no_html if words not in pucn]
    words_wo_punct=''.join(no_punct)
    words_wo_punct = words_wo_punct.lower()
    emoji = re.compile("["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF" 
        u"\U00002500-\U00002BEF"  
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030""]+", re.UNICODE)
    no_emoji = re.sub(emoji, '', words_wo_punct)
    return no_emoji

In [ ]:
comments = list(df['text'])

In [ ]:
len(comments)

In [ ]:
test_sentence = comments[200]

In [ ]:
test_sentence

In [ ]:
cleaned_text = clean_sentences(test_sentence)

In [ ]:
cleaned_text

In [ ]:
X =[]
for sentence  in comments:
    X.append(clean_sentences(sentence))

In [ ]:
X[200]

In [ ]:
emotions

In [ ]:
len(emotions)

In [ ]:
y = df[[
 'admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']].values

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(X_train) # create word to index relationship-each word is given a unique index mostly according to appearance in the corpus

vocab_size = len(tokenizer.word_index) + 1 #size of the total number of uniques tokens in the dataset

tokenized_train = tokenizer.texts_to_sequences(X_train) #converting tokens into sequence of integers
tokenized_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train[10]

In [ ]:
tokenized_train[10]

In [ ]:
max_len = 100

padded_train = pad_sequences(tokenized_train, maxlen=max_len, padding='pre') 

padded_test = pad_sequences(tokenized_test, maxlen=max_len, padding='pre')

In [ ]:
padded_train[10]

In [ ]:
y[10]

In [ ]:
vocab_size #all unique tokens in the corpus

In [ ]:

"""
create an embedding matrix with glove words  and vectors 
- each word is a key and a corresponding vector is the value
"""
emmbeddings_dictionary = {}

glove_model =  open('glove.6B.100d.txt', encoding='utf8')
    
for line in glove_model:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbeddings_dictionary[word]=vector
    
glove_model.close()    

In [ ]:
"""
create an embedding matrix for the corpus where each word in the corpus is a key and the corresponding vector 
of a dimension of 100 is the value
each row contains a number which corresponds to the index of the word in the corpus
the matrix has 100 columns-each column contains word embedding for each word in the corpus
"""

embedding_dim=100
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, index in tokenizer.word_index.items():
    embedding_vector = emmbeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index]=embedding_vector


In [ ]:
embedding_matrix.shape

In [ ]:
model = Sequential() #initialize sequential model

# create embedding layer
embedding_layer = Embedding(vocab_size, embedding_dim,
                            weights=[embedding_matrix], input_length=max_len, trainable=False)

# add layers to the model
model.add(embedding_layer)
model.add(LSTM(256, return_sequences = True))
# model.add(Dropout(0.2))
model.add(LSTM(128))
# model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(28, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history=model.fit(padded_train, y_train, epochs=200, batch_size=128, validation_split=0.3)

In [ ]:
y_pred = model.predict(padded_test)

In [ ]:
model.save('EmoText')